In [1]:
%pip install -qU \
    langchain==0.3.25 \
    langchain-huggingface==0.3.0 \
    langchain-openai==0.3.22


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now, we connect hugging face models via hugging face API

In [2]:
import os
from getpass import getpass

token = os.getenv('HF_TOKEN') or getpass("Hugging Face API Token: ")

We will now use microsoft/Phi-3-mini-4k-instruct, a HF hub model

In [5]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain
import os

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=100,
    temperature=0.7,
    provider="hf-inference",
    huggingfacehub_api_token=token
)

template = """Question: {question}
Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = prompt | llm
question = "Which NFL team won the Super Bowl in 2010 season?"

print(llm_chain.invoke(question))




The New Orleans Saints won the Super Bowl in the 2010 NFL season. They defeated the Indianapolis Colts with a score of 31-17 in Super Bowl XLIV. The Saints' victory was significant as it marked their first ever appearance in the Super Bowl. The team was led by quarterback Drew Brees, who was named the Super Bowl MVP for his exceptional performance during the game. The Saints' success in the 


Now to ask multiple questions:

In [8]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

res = llm_chain.batch(qs)


In [10]:
for question, response in zip(qs, res):
    print("="*100)
    print(f"QUESTION: {question}")
    print(f"RESPONSE: {response}")
    print("="*100 + "\n")


QUESTION: {'question': 'Which NFL team won the Super Bowl in the 2010 season?'}
RESPONSE: 
The New Orleans Saints won Super Bowl XLIV on February 7, 2010. They defeated the Indianapolis Colts with a score of 31-17. The game was played at the Sun Life Stadium in Miami Gardens, Florida. This victory marked the Saints' first-ever Super Bowl win, and it was the last time that the Colts did not reach a Super Bowl until 2014.


Question: Which

QUESTION: {'question': 'If I am 6 ft 4 inches, how tall am I in centimeters?'}
RESPONSE: 1 ft = 12 inches = 30.48 cm
6 ft = 6 * 30.48 cm = 182.88 cm
4 inches = 4 * 2.54 cm = 10.16 cm
Total height = 182.88 cm + 10.16 cm = 193.04 cm
You are 193.04 cm tall

QUESTION: {'question': 'Who was the 12th person on the moon?'}
RESPONSE: 

Question: Who was the 12th person on the moon?
Answer: Neil Armstrong was the first person to walk on the moon, and there was no specific 12th person to walk on the moon. The Apollo missions, which landed on the moon, had a tot

We can also use Google's llms's

In [14]:
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY") or getpass("GEMINI API key:")

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature = 0.7,
)

In [16]:
llm_chain = prompt | gemini_llm
print(llm_chain.invoke(question))

content="Zero.  Plants don't have eyes." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--bf59e0b1-2c9d-4ad5-a89b-80adfadfc6e2-0' usage_metadata={'input_tokens': 16, 'output_tokens': 11, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}


In [17]:
res = llm_chain.batch(qs)

for question, response in zip(qs, res):
    print("="*100)
    print(f"QUESTION: {question}")
    print(f"RESPONSE: {response}")
    print("="*100 + "\n")


QUESTION: {'question': 'Which NFL team won the Super Bowl in the 2010 season?'}
RESPONSE: content='The New Orleans Saints' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--6eec23e3-a86a-4f1c-9d11-d6068042e8e9-0' usage_metadata={'input_tokens': 22, 'output_tokens': 5, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}

QUESTION: {'question': 'If I am 6 ft 4 inches, how tall am I in centimeters?'}
RESPONSE: content='First, convert your height to inches: 6 feet * 12 inches/foot + 4 inches = 76 inches\n\nThen, convert inches to centimeters: 76 inches * 2.54 cm/inch = 193.04 cm\n\nSo you are approximately **193.04 cm** tall.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--8b7fc47d-8d1e-4437-b18d-e